# Line Following Robot CoppeliaSim

### Setup

Make sure to have the server side running in CoppeliaSim: 
- in a child script of a CoppeliaSim scene, add following command to be executed just once, at simulation start:

```
simRemoteApi.start(19999)
```
then start simulation, and run this program.

IMPORTANT: for each successful call to simxStart, there
should be a corresponding call to simxFinish at the end!

In [1]:
%matplotlib qt

try:
    import sim
except:
    print ('--------------------------------------------------------------')
    print ('"sim.py" could not be imported. This means very probably that')
    print ('either "sim.py" or the remoteApi library could not be found.')
    print ('Make sure both are in the same folder as this file,')
    print ('or appropriately adjust the file "sim.py"')
    print ('--------------------------------------------------------------')
    print ('')

import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

We have instructed CoppeliaSim to start a communication server on port 19999. The following statement is used to establish communication

In [2]:
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim in synchronous

if clientID!=-1:
    print ('Connected to remote API server')
else:
    print ('Failed connecting to remote API server')  

Connected to remote API server


### Robot Initialization

On the first part of the script, we need to get the joint objects, sensor objects and set their operation modes.

- Object handles are fetched using the blocking mode
- The first call to simxReadVisionSensor() should be "streaming" and the consecutive calls should be "buffer" mode

In [3]:
# Motor handles
motor = [-1, -1, -1, 1]  # motor[0] motor[1] are left, motor[2] motor[3] are right

# Sensor handles
camHandle = -1

# Now try to retrieve handles in a blocking fashion (i.e. a service call):

if clientID!=-1:

    # ---------------------Initialize Motor--------------------- #
    
    err, motor[0] =sim.simxGetObjectHandle(clientID, '/Bot_Red/Motor_FL', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error getting motor 1")
    err, motor[1] =sim.simxGetObjectHandle(clientID, '/Bot_Red/Motor_BL', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error getting motor 2")
    err, motor[2] =sim.simxGetObjectHandle(clientID, '/Bot_Red/Motor_FR', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error getting motor 3")
    err, motor[3] =sim.simxGetObjectHandle(clientID, '/Bot_Red/Motor_BR', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error getting motor 4")
    
    # ---------------------Initialize Sensors--------------------- #
    
    err, camHandle = sim.simxGetObjectHandle(clientID, '/Bot_Red/Cam', sim.simx_opmode_blocking)
    if(err != 0):
        print("Error_camera")    
     
    # ---------------------Enable Streaming--------------------- #
    
    err, res, image = sim.simxGetVisionSensorImage(clientID,  camHandle, 0, sim.simx_opmode_streaming)
    
    print("Done Initialization")

    time.sleep(2)
    
else:
    print ('Failed connecting to remote API server')

Done Initialization


### Control parameters

In this part, we set up some functions to control the robot

- a function for reading sensor easily
- a function for controlling motor speeds easily

In [4]:
def readSensor():
        
    # Try to retrieve the streamed image
    
    err, res, image_raw = sim.simxGetVisionSensorImage(clientID,  camHandle, 0, sim.simx_opmode_buffer) 
    if(err != 0):
        print("Error getting image") 
    # print(err)
    # print(res)
    img = np.array(image_raw, dtype=np.uint8)
    img.resize([res[1], res[0], 3])
    img = img[::-1,:, :]
    return img 

In [5]:
def motorControl(leftV, rightV):

    # Two of the left motors use left velocity, two of the right motors use right velocity
    sim.simxSetJointTargetVelocity(clientID, motor[0], leftV, sim.simx_opmode_oneshot)
    sim.simxSetJointTargetVelocity(clientID, motor[1], leftV, sim.simx_opmode_oneshot)
    sim.simxSetJointTargetVelocity(clientID, motor[2], rightV, sim.simx_opmode_oneshot)
    sim.simxSetJointTargetVelocity(clientID, motor[3], rightV, sim.simx_opmode_oneshot)

### Empty Control Routine

- Get readings from sensor
- Complete necessary logical/mathematical operations to detect line
- Send speed commands to motor

In [22]:
minSpeed = 10*np.pi/180
maxSpeed = 50*np.pi/180

speed = (minSpeed + maxSpeed)/2

leftV = 0
rightV = 0



if clientID!=-1:
    print ("Running Dummy Routine")
    
    startTime = time.time()
    
    # Loop for 10 seconds
    while time.time()-startTime < 5: 
        
        # ----------------------------------- Get sensor data ----------------------------------- #
        image = readSensor()
        # print(image.shape)
        
        # ----------------------------------- Analyze data/Decision ----------------------------------- #

        cv2.startWindowThread()
        cv2.namedWindow("image")
        cv2.imshow('image', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        # ----------------------------------- Send motor command ----------------------------------- #

        motorControl(0, 0)
        
    # stop motors
    motorControl(0, 0)
        
else:
    print ('Failed connecting to remote API server')

Running Soccer Routine


### Ball Tracking Routine

In [6]:
def find_ball(img):
    
    # convert to HSV colorspace and find the orange regions
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    orange_mask = cv2.inRange(hsv, np.array([5,10,10]), np.array([20,255,255]))
    orange_parts = cv2.bitwise_and(img, img, mask= orange_mask)
    
    # filter out random pixels
    kernel = np.ones((5,5),np.uint8)
    opening_orange = cv2.morphologyEx(orange_mask, cv2.MORPH_OPEN, kernel)

    overlay_image = np.copy(img)

    # Orange contour
    contours = cv2.findContours(opening_orange, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]

    centroid = None
    
    for i in contours:
        y,x,w,h = cv2.boundingRect(i)
        cv2.rectangle(overlay_image, (y, x), (y + w, x + h), (0,255,255), 4)
        centroid = np.array([x+h/2, y+w/2]).astype(float)

    return overlay_image, centroid

In [7]:
def randomize_ball():
    
    # in this function, we set random positions to the ball
    
    err, ball = sim.simxGetObjectHandle(clientID, '/Field/Ball', sim.simx_opmode_blocking)
    
    err, pos  = sim.simxGetObjectPosition(clientID, ball, -1, sim.simx_opmode_blocking)
    
    pos[0] = (np.random.rand()-0.5)/2
    pos[1] = np.random.rand()-0.5
    
    err = sim.simxSetObjectPosition(clientID, ball, -1, pos, sim.simx_opmode_oneshot)

In [93]:
minSpeed = 10*np.pi/180
maxSpeed = 50*np.pi/180

speed = (minSpeed + maxSpeed)/2

leftV = 0
rightV = 0

# We will set the ball at 5 random positions and track it

for i in range(5):
    
    if clientID!=-1:
        
        print ("Randomizing Ball Location")
        randomize_ball()

        print ("Trying to locate and track ball")
        startTime = time.time()

        # Loop for 10 seconds to find ball and center it on screen
        while time.time()-startTime < 10: 

            # ----------------------------------- Get sensor data ----------------------------------- #
            image = readSensor()

            # ----------------------------------- Analyze data/Decision ----------------------------------- #

            # converting RGB image to BGR, BGR is the default for opencv
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            overlay_image, centroid = find_ball(image)
            
            # deciding whether ball is on left or right. 
            # If ball is not in frame, keep rotating until it is found
            if centroid is None:
                leftV = maxSpeed
                rightV = -maxSpeed
            else:
                horizontal_displacement = (image.shape[1]/2 - centroid[1]) / image.shape[1]
                # negative displacement means ball is on the right
                # positive displacement means ball is on the left
                leftV = -horizontal_displacement
                rightV = horizontal_displacement
                
            cv2.startWindowThread()
            cv2.namedWindow("Tracking ball")
            cv2.imshow('Tracking ball', overlay_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            # ----------------------------------- Send motor command ----------------------------------- #

            motorControl(leftV, rightV)

        # stop motors
        motorControl(0, 0)

    else:
        print ('Failed connecting to remote API server')


Randomizing Ball Location
Trying to locate and track ball
Randomizing Ball Location
Trying to locate and track ball
Randomizing Ball Location
Trying to locate and track ball
Randomizing Ball Location
Trying to locate and track ball
Randomizing Ball Location
Trying to locate and track ball


### Catch the Ball

In [9]:
minSpeed = 10*np.pi/180
maxSpeed = 50*np.pi/180

speed = (minSpeed + maxSpeed)/2

leftV = 0
rightV = 0

# We will set the ball at 5 random positions and track it

for i in range(5):
    
    if clientID!=-1:
        
        print ("Randomizing Ball Location")
        randomize_ball()

        print ("Trying to locate and track ball")
        startTime = time.time()

        # Loop for 10 seconds to find ball and center it on screen
        while time.time()-startTime < 10: 

            # ----------------------------------- Get sensor data ----------------------------------- #
            image = readSensor()

            # ----------------------------------- Analyze data/Decision ----------------------------------- #

            # converting RGB image to BGR, BGR is the default for opencv
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            overlay_image, centroid = find_ball(image)
            
            # deciding whether ball is on left or right. 
            # If ball is not in frame, keep rotating until it is found
            if centroid is None:
                leftV = maxSpeed
                rightV = -maxSpeed
            else:
                horizontal_displacement = (image.shape[1]/2 - centroid[1]) / image.shape[1]
                # negative displacement means ball is on the right
                # positive displacement means ball is on the left
                
                vertical_displacement = (image.shape[0] - centroid[0]) / image.shape[0]
                # the higher displacement, further away the ball
                
                leftV = -horizontal_displacement + vertical_displacement*2
                rightV = horizontal_displacement + vertical_displacement*2
                   
            cv2.startWindowThread()
            cv2.namedWindow("Tracking ball")
            cv2.imshow('Tracking ball', overlay_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            # ----------------------------------- Send motor command ----------------------------------- #

            motorControl(leftV, rightV)

        # stop motors
        motorControl(0, 0)

    else:
        print ('Failed connecting to remote API server')


Randomizing Ball Location
Trying to locate and track ball
Randomizing Ball Location
Trying to locate and track ball
Randomizing Ball Location
Trying to locate and track ball
Randomizing Ball Location
Trying to locate and track ball
Randomizing Ball Location
Trying to locate and track ball


### Terminate simulation

After simulation ends, we need to properly disconnect from the simulation

In [ ]:
sim.simxFinish(clientID)

    
print ('Program ended')